# Primer3-MCP Demo: PCR Primer Design

This notebook demonstrates how to use primer3-mcp as a Python library for primer design and analysis.

## Setup

Import the library functions we'll use:

In [1]:
from primer3_mcp import design_primers, troubleshoot_primers
import pandas as pd

## Example 1: Basic Primer Design

Design primers for a target region marked with `[n]` placeholder.

In [11]:
# Example sequence with [n] marking the target region
sequence = "GGGTCAGGTAGGAACGCGTGCCAGATCTGTCTCAACAAGAAAGAGGACCCTGAAACTGACTGTTTTACCGCTGTCTCAGAAGTCTTGAAAATGTCTATCATGAATGCTGTTAAGATATCTATAGGCTGGCGGGCACAATCAGTAGTTAATGCTGCCATTGATAAC[n]ACTAGTGATCAACCTCTGAAGAGGTCAACCATGCCATATCCATGTGATGATGGCTGGTGCCTTGATAGTGACAACGTTACTTACAGCACTCCTGATCTGTATACGCACGCAGAGCAGTTATACGCAGGTACGGATGTGGTAATACTAACACCGGAATCACTGACA"

# Design primers with default parameters
result = design_primers(sequence=sequence, num_return=5)

# Print summary
print(result)

Primer Design Results: 5 pairs found


## Example 2: Accessing Primer Details

Explore the structured data returned by the design function.

In [5]:
# Get the best primer pair
best_pair = result.pairs[0]

print(f"Pair ID: {best_pair.pair_id}")
print(f"Product Size: {best_pair.product_size}bp")
print(f"Penalty Score: {best_pair.penalty:.3f}")
print()

# Left primer details
print("Left Primer:")
print(f"  Sequence: {best_pair.left_primer.sequence}")
print(f"  Tm: {best_pair.left_primer.tm:.1f}°C")
print(f"  GC%: {best_pair.left_primer.gc_percent:.1f}%")
print(f"  Length: {best_pair.left_primer.length}bp")
print()

# Right primer details
print("Right Primer:")
print(f"  Sequence: {best_pair.right_primer.sequence}")
print(f"  Tm: {best_pair.right_primer.tm:.1f}°C")
print(f"  GC%: {best_pair.right_primer.gc_percent:.1f}%")
print(f"  Length: {best_pair.right_primer.length}bp")

Pair ID: 0
Product Size: 171bp
Penalty Score: 0.743

Left Primer:
  Sequence: GGCGGGCACAATCAGTAGTTAATGC
  Tm: 64.6°C
  GC%: 52.0%
  Length: 25bp

Right Primer:
  Sequence: CCGTACCTGCGTATAACTGCTCTGC
  Tm: 65.4°C
  GC%: 56.0%
  Length: 25bp


## Example 3: Custom Parameters

Design primers with custom size and temperature ranges.

In [6]:
# Design with custom parameters
custom_result = design_primers(
    sequence=sequence,
    primer_size_min=18,
    primer_size_opt=22,
    primer_size_max=27,
    primer_tm_min=60.0,
    primer_tm_opt=62.0,
    primer_tm_max=64.0,
    gc_clamp=1,
    num_return=3
)

print(f"Found {custom_result.num_returned} primer pairs with custom parameters")
print()

# Show all pairs
for pair in custom_result.pairs:
    print(pair)
    print()

Found 3 primer pairs with custom parameters

Pair 0: 178bp product | Penalty: 0.29
  Left:  ATATCTATAGGCTGGCGGGCAC (Tm: 61.8°C)
  Right: CTGCGTATAACTGCTCTGCGTG (Tm: 61.9°C)

Pair 1: 166bp product | Penalty: 0.72
  Left:  ATATCTATAGGCTGGCGGGCAC (Tm: 61.8°C)
  Right: GCTCTGCGTGCGTATACAGATC (Tm: 61.5°C)

Pair 2: 180bp product | Penalty: 0.81
  Left:  AGATATCTATAGGCTGGCGGGC (Tm: 61.3°C)
  Right: CTGCGTATAACTGCTCTGCGTG (Tm: 61.9°C)



## Example 4: Data Analysis with Pandas

Convert results to DataFrame for easy analysis and filtering.

In [7]:
# Convert to DataFrame
df = result.to_dataframe()

# Display key columns
df[['pair_id', 'product_size', 'penalty', 'left_tm', 'right_tm', 'left_gc', 'right_gc']]

,pair_id,product_size,penalty,left_tm,right_tm,left_gc,right_gc
0,0,171,0.742571,64.626977,65.369548,52.0,56.0
1,1,153,0.783622,64.626977,65.410598,52.0,56.0
2,2,115,0.830243,64.626977,65.457220,52.0,52.0
3,3,170,0.956241,64.626977,65.583217,52.0,56.0
4,4,148,0.994126,64.626977,65.621103,52.0,56.0


In [8]:
# Filter primers by criteria
good_primers = df[
    (df['penalty'] < 1.0) &
    (df['product_size'] > 200) &
    (df['left_tm'].between(64, 66)) &
    (df['right_tm'].between(64, 66))
]

print(f"Found {len(good_primers)} primer pairs meeting strict criteria")
good_primers[['pair_id', 'product_size', 'penalty', 'left_tm', 'right_tm']]

Found 0 primer pairs meeting strict criteria


,pair_id,product_size,penalty,left_tm,right_tm


In [9]:
# Analyze primer characteristics
print("Primer Statistics:")
print(f"Average product size: {df['product_size'].mean():.1f}bp")
print(f"Average left Tm: {df['left_tm'].mean():.1f}°C")
print(f"Average right Tm: {df['right_tm'].mean():.1f}°C")
print(f"Average GC% (left): {df['left_gc'].mean():.1f}%")
print(f"Average GC% (right): {df['right_gc'].mean():.1f}%")

Primer Statistics:
Average product size: 151.4bp
Average left Tm: 64.6°C
Average right Tm: 65.5°C
Average GC% (left): 52.0%
Average GC% (right): 55.2%


## Example 5: Troubleshooting Difficult Sequences

Use auto-retry with progressive constraint relaxation for difficult sequences.

In [16]:
# Difficult sequence (AT-rich, might need relaxed constraints)
difficult_seq = "GCGCGCATATATATGCGCGCATATATATGCGCGC[n]ATATATATGCGCGCATATATATGCGCGCATATAT"

# Try troubleshooting mode
troubleshoot_result = troubleshoot_primers(
    sequence=difficult_seq,
    num_return=5
)

print(troubleshoot_result)
print()

if troubleshoot_result.num_returned > 0:
    print("Success! Primers found.")
    if troubleshoot_result.troubleshooting_applied:
        print(f"Troubleshooting steps applied: {troubleshoot_result.troubleshooting_applied}")
    print()
    print("Best pair:")
    print(troubleshoot_result.pairs[0])
else:
    print("No primers found even with relaxed constraints.")

Primer Design Results: 0 pairs found
Troubleshooting: Reduced GC clamp to 1; Reduced GC clamp to 0; Widened Tm range by ±1°C; No primers found - consider wider sequence search area

No primers found even with relaxed constraints.


## Example 6: Batch Processing Multiple Sequences

Process multiple sequences and compare results.

In [18]:
# Multiple sequences to analyze
sequences = {
    "Gene A": "GGGTCAGGTAGGAACGCGTGCCAGATCTGTCTCAACAAGAAAGAGGACCCTGAAACTGACTGTTTTACCGCTGTCTCAGAAGTCTTGAAAATGTCTATCTTGAATGCTGTTAAGATATCTATAGGCTGGCGGGCACAATCAGTAGTTAATGCTGCCATTGATAAC[n]ACTAGTGATCAACCTCTGAAGAGGTCAACCATGCCATATCCATGTGATGATGGCTGGTGCCTTGATAGTGACAACGTTACTTACAGCACTCCTGATCTGTATACGCACGCAGAGCAGTTATACGCAGGTACGGATGTGGTAATACTAACACCGGAATCACTGACA",
    "Gene B": "CCGTACGTACGTACGGGTCAGGTAGGAACGCGTGCCAGATCTGTCTCAACAAGAAAGAGGACCCTGAAACTGACTGTTTTACCGCTGTCTCAGAAGTCTTGAAAATGTCTATCATGAATGCTGTTAAGATATCTATAGGCTGGCGGGCACAATCAGTAGTTAATGCTGCCATTGATAAC[n]ACTAGTGATCAACCTCTGAAGAGGTCAACCATGCCATATCCATGTGATGATGGCTGGTGCCTTGATAGTGACAACGTTACTTACAGCACTCCTGATCTGTATACGCACGCAGCAGTTATACGCAGGTACGGATGTGGTAATACTAACACCGGAATCACTGACA",
    "Gene C": "ATGCATGCATGCATGCATGGTCAGGTAGGAACGCGTGCCAGATCTGTCTCAACAAGAAAGAGGACCCTGAAACTGACTGTTTTACCGCTGTCTCAGAAGTCTTGAAAATGTCTATCATGAATGCTGTTAAGATATCTATAGGCTGGCGGGCACAATCAGTAGTTAATGCTGCCATTGATAAC[n]ACTAGTGATCAACCTCTGAAGAGGTCAACCATGCCATATCCATGTGATGATGGCTGGTGCCTTGATAGTGACAACGTTACTTACAGCACTCCTGATCTGTATACGCACGCAGAGCAGTTATACGCAGGTACGGATGTGGTAATACTAACACCGGAATCACTGACAATGCTAAAAAAAAAAAAAA",
}

# Process all sequences
results = {}
for name, seq in sequences.items():
    try:
        results[name] = design_primers(sequence=seq, num_return=3)
        print(f"{name}: {results[name].num_returned} pairs found")
    except Exception as e:
        print(f"{name}: Error - {e}")

Gene A: 3 pairs found
Gene B: 3 pairs found
Gene C: 3 pairs found


In [19]:
# Combine all results into one DataFrame
all_results = []
for name, result in results.items():
    df_temp = result.to_dataframe()
    df_temp['gene'] = name
    all_results.append(df_temp)

combined_df = pd.concat(all_results, ignore_index=True)

# Show summary by gene
combined_df.groupby('gene').agg({
    'pair_id': 'count',
    'product_size': 'mean',
    'penalty': 'min',
    'left_tm': 'mean',
    'right_tm': 'mean'
}).round(2)

,pair_id,product_size,penalty,left_tm,right_tm
gene,,,,,
Gene A,3,146.33,0.74,64.63,65.41
Gene B,3,144.67,0.45,64.63,65.33
Gene C,3,210.33,0.74,64.88,65.38


## Example 7: Exporting Results

Export primer designs to CSV for ordering or further analysis.

In [20]:
# Create a simple ordering sheet
ordering_df = df[['pair_id', 'left_sequence', 'right_sequence', 'product_size', 'left_tm', 'right_tm']].copy()
ordering_df.columns = ['Pair', 'Forward Primer (5\'→3\')', 'Reverse Primer (5\'→3\')', 'Product (bp)', 'Fwd Tm (°C)', 'Rev Tm (°C)']

print("Primer Ordering Sheet:")
ordering_df

Primer Ordering Sheet:


,Pair,Forward Primer (5'→3'),Reverse Primer (5'→3'),Product (bp),Fwd Tm (°C),Rev Tm (°C)
0,0,GGCGGGCACAATCAGTAGTTAATGC,CCGTACCTGCGTATAACTGCTCTGC,171,64.626977,65.369548
1,1,GGCGGGCACAATCAGTAGTTAATGC,GCTCTGCGTGCGTATACAGATCAGG,153,64.626977,65.410598
2,2,GGCGGGCACAATCAGTAGTTAATGC,ACGTTGTCACTATCAAGGCACCAGC,115,64.626977,65.457220
3,3,GGCGGGCACAATCAGTAGTTAATGC,CGTACCTGCGTATAACTGCTCTGCG,170,64.626977,65.583217
4,4,GGCGGGCACAATCAGTAGTTAATGC,GCGTGCGTATACAGATCAGGAGTGC,148,64.626977,65.621103


In [ ]:
# Save to CSV (uncomment to actually save)
# ordering_df.to_csv('primer_orders.csv', index=False)
# print("Saved to primer_orders.csv")